In [12]:
from typing import Tuple
import pandas as pd 
import random 
import seaborn as sns

In [13]:
class ProcessingData:
    # zwracam dataframe w losowej kolejności i resetuje indeksy
    @staticmethod
    def shuffleDF(df: pd.DataFrame) -> pd.DataFrame:
        return df.iloc[random.sample(range(len(df)), len(df))].reset_index(drop=True)

    # normalizuje każdą kolumnę w dataframe (według przekazanej listy) metodą min max
    @staticmethod
    def normalizeDF(df: pd.DataFrame, columnNames: list) -> pd.DataFrame:
        for columnName in columnNames:
            df[columnName] = (df[columnName]-df[columnName].min())/(df[columnName].max()-df[columnName].min())
        return df

    # zwracam wiersze gdzie index jest mniejszy od długość df*wielkość df i wiersze gdzie index jest równy lub większy od długość *wielkość df 
    @staticmethod
    def splitDF(df: pd.DataFrame, trainSize: float) -> Tuple[pd.DataFrame, pd.DataFrame]:
        return df[df.index < int(len(df)*trainSize)], df[df.index >= int(len(df)*trainSize)]

    @staticmethod
    def processData(df: pd.DataFrame, columnNames: list, trainSize: float) -> Tuple[pd.DataFrame, pd.DataFrame]:
        df = ProcessingData.shuffleDF(df)
        df = ProcessingData.normalizeDF(df, columnNames)
        return ProcessingData.splitDF(df, trainSize)

In [14]:

class KNN:
    def __init__(self, k: int):
        self.k = k
    
    #liczę dystatns przy użyciu metryki taksówkowej
    @staticmethod
    def dst(x: list, y: list) -> float:
        return sum([(abs(xi-yi))
                    for xi,yi in zip(x,y)])

    #do obiektu wpisuje dane treningowe
    def fit(self, df: pd.DataFrame):
        self.df = df
    
    #dla danego obiektu zwraca najczęściej pojawianego się sąsiada
    def predict(self, point: pd.DataFrame) -> str:
        #tworzę słownik z typami
        types = {}
        for v in pd.unique(self.df["Type"]):
            types[v] = 0
        result = []
        # liczę odległości od każdego punktu
        for sample in self.df.values:
            result.append([KNN.dst(sample[:-1], point), sample[-1]])
        #sortuje odległości
        result.sort(key=lambda x:x[0])
        # zliczam type k najbliższych odległości
        for i in range(self.k):
            types[result[i][1]] += 1
        return max(types, key=types.get)
    
    #liczę wynik dla zbioru testowego
    def score(self, test_X: pd.DataFrame) -> float:
        good = 0
        bad = 0
        for sample in test_X.values:
            # sprawdzam czy predict zwrócił dobry typ
            if (x:=self.predict(sample)) == sample[-1]:
                good += 1
            else:
                bad +=1
        return good/(bad+good)*100

In [18]:
# znormalizowane
seeds = pd.read_csv("seeds.csv", sep=",")
X_train, X_test = ProcessingData.processData(seeds, seeds.columns[:-1], 0.8)
knn = KNN(4)
knn.fit(X_train)
print(knn.score(X_test))

97.5


In [38]:
# bez normalizacji
seeds = pd.read_csv("seeds.csv", sep=",")
seeds = ProcessingData.shuffleDF(seeds)
X_train, X_test = ProcessingData.splitDF(seeds, 0.8)
knn = KNN(4)
knn.fit(X_train)
print(knn.score(X_test))

90.0
